### 유튜브 수집

- 대표적인 영상 서비스
- 구글 제공
- 특정 현상, 주제, 관심 등에 대한 다양한 영상과 댓글들이 전 세계적으로 발생하여, 이를 통해 해당 주제에 대한  여론을 예측
- 방법
> 기본 파일럿으로 오늘데이터만 수집
> 접속방법
> - https://www.youtube.com/results?search_query=bts&sp=EgQIAhAB
> - 썸네일, 링크, 제목, 작성자, 조회수 및 시간은 검토해서 세부 정보에서 좀 더 디테일하게 나오면 수집
> - 세부 영상 정보로 가서는 좋아요, 싫어요, 작성일(시간), 작성자 정보 최대한, 댓글(작성자, 시간, 내용) 수집 
- 사용 솔루션
> - selenium 사용 : ajax 통신 사용(스크롤 이벤트 처리)
> - 고스트 브라우저를 활용하여 리눅스 운영 검토
> - 디비 입력 처리(차후 자동화 -> 장고의 모델 처리 방식 응용(orm 방식 처리) )
> - GUI 솔루션 형태 검토 (클라이언트 배포판) <-> API 서버가 존재해야함(Flask 등장)

In [1]:
from selenium import webdriver as wd

In [33]:
# 기본 접속
driver = wd.Chrome('./data/chromedriver.exe')
filters = {'hour':'EgQIARAB', # 지난 한시간
          'today':'EgQIAhAB', # 오늘
          'week':'EgQIAxAB',  # 이번주
          'month':'EgQIBBAB',  # 이번달
          'year':'EgQIBRAB'}  # 올해
word = 'bts'
url = 'https://www.youtube.com/results?search_query={word}&sp={date}'.format(word=word,date=filters['today'])

In [34]:
# 접속
driver.get(url)

In [35]:
# 데이터를 다 불러 온다. -> 스크롤 바를 맨 밑으로 이동 -> javascript 사용 
# 스크롤바를 내리고 데이터가 로드되었는데, 데이터 개수가 이전 데이터 개수와 동일 하다 -> 끝
# 현재 화면에 보이는 ytd-video-renderer의 총 개수는?
# 태그명 ytd-video-renderer를 찾는다.

def getVideoCount():
    return len(driver.find_elements_by_css_selector('ytd-video-renderer'))
getVideoCount()

17

In [36]:
# 단순하게 10바퀴 돌리면서 스크롤 쳐본다.
import time

# for n in range(10):
#     # 스크롤바를 맨밑으로 이동 -> javascript 사용
#     driver.execute_script('''
#     window.scrollBy(0, 1000)
#     ''')
#     data_number = getVideoCount()
#     time.sleep(2)

In [37]:
# data_number = 0
# temp = 1 
# while True:     
#     if data_number != temp:
#         temp = data_number  # 이전
#         print('temp:',temp)
#         driver.execute_script('''
#     window.scrollBy(0, 1000)
#     ''')        
#         time.sleep(2)
#         data_number = getVideoCount()  # 업데이트
#         print('data_number:',data_number)
#     else:
#         break

#### time.sleep은 전체기능 자체가 멈춰져서 명확한코드를 짜기가 애매하다.
# 문제점 분석해보기

In [38]:
# 시도하기전 총 개수를 담는 변수
video_cnt = getVideoCount() # 최초 20개
# 시도 회수를 담는 변수
try_cnt = 0
# 선언 초기화 업데이트 사용
# flag 변수
isCollecting = True
n = 0
while isCollecting:
    n += 1
    try_cnt += 1 # 반복 횟수
    driver.execute_script('''
    window.scrollBy(0, 1000) 
    ''')# 무한 스크롤이 실행되는 시점에서 scrolling이 멈출 것이라는 보장이 없다. 아마도 추측엔 숫자가 맞지 않아서 엉키는 문제가 발생할 거라 생각함.
    time.sleep(1)    
    if video_cnt == getVideoCount() and try_cnt<10: # 좌측(이전 비디오 갯수) == 우측(현재 비디오 갯수)
        continue
    elif video_cnt != getVideoCount():
        print('총 %d 번째 시도, 개별 시도 %d번째, 총 개수%d '%(n, try_cnt,getVideoCount()))
        # 현재 총 개수 설정
        video_cnt = getVideoCount()  # 업데이트
        #시도 횟수 초기화
        try_cnt = 0
    else:
        #&&는 앞 부분이 에러 나면 바로 아래로 감.
        print('수집완료')
        break
   
   

총 3 번째 시도, 개별 시도 3번째, 총 개수40 
총 8 번째 시도, 개별 시도 5번째, 총 개수60 
총 12 번째 시도, 개별 시도 4번째, 총 개수80 
총 15 번째 시도, 개별 시도 3번째, 총 개수99 
총 18 번째 시도, 개별 시도 3번째, 총 개수119 
총 21 번째 시도, 개별 시도 3번째, 총 개수139 
총 24 번째 시도, 개별 시도 3번째, 총 개수159 
총 27 번째 시도, 개별 시도 3번째, 총 개수179 
총 31 번째 시도, 개별 시도 4번째, 총 개수199 
총 34 번째 시도, 개별 시도 3번째, 총 개수215 
총 37 번째 시도, 개별 시도 3번째, 총 개수222 
수집완료


In [8]:
# # 기본 메타 정보 수집
# # 썸네일, 링크, 제목, 작성자, 조회수 및 시간은 검토해서 세부 정보에서 좀 더 디테일하게 나오면 수집
# def getVideoElement():
#     return driver.find_elements_by_css_selector('ytd-video-renderer')
# # 유튜브 정보를 담는 클래스(그릇역할)->향후 orm을 으로 연결되어 디비에 자동 삽입되는 형태로 변환
# #BTS perform Idol!!! - BBC 게시자: BBC 2일 전 4분, 1초 조회수 1,965,185회
# class YouTubeVideoMode:
#     link = None
#     title = None
#     author = None
#     reviewCnt = None
#     playtime = None
#     thum = None
# for video in getVideoElement():
#     du = video.find_element_by_css_selector('.style-scope ytd-video-meta-block').text.split('\n')
#     # 링크
#     print(video.find_element_by_id('thumbnail').get_attribute('href'))
#     # 제목
#     print(video.find_element_by_id('video-title').get_attribute('aria-label').split('게시자')[0])
#     # 썸네일
#     print(video.find_element_by_css_selector('.style-scope yt-img-shadow')
#     # 작성자
#     print(video.find_element_by_css_selector('.yt-simple-endpoint style-scope yt-formatted-string').text)
#     # 조회수
#     print(du[2])
#     # 플레이 시간

In [25]:
def getVideoElement():
    return driver.find_elements_by_css_selector('ytd-video-renderer')

In [26]:
class YouTubeVideoModel:
    # 영상 콘텐츠 링크-> 고유값 https://www.youtube.com/watch?v=(생략)  //4TLMNQR25o
    link = None
    # 제목 
    title = None
    # 작성자
    author = None
    # 작성자홈  https://www.youtube.com/channel/ 생략
    author_addr = None
    # 조회수
    reviewCnt = 0
    # 재생시간
    playtime = None
    # 썸네일
    thum = None
    
    # 게시일, 조회수, 좋아요, 싫어요, 영상 설명8
    
    
    # 게시일
    startDay = None
    # 좋아요 수
    likes = None
    # 싫어요 수
    hates = None   
    # 영상 설명
    comment = None
    # 카테고리
    cate = None
    
    
    
    def __repr__(self):
        return '''
        link =%s, 
        title=%s, 
        author=%s, 
        author_addr=%s, 
        reviewCnt=%s, 
        playtime=%s, thum=%s
        '''% (self.link, self.title, self.author, self.author_addr, self.reviewCnt, self.playtime, self.thum)

In [27]:
# 기본 메타 정보 수집
# 썸네일, 링크, 제목, 작성자, 조회수 및 시간은 검토해서 세부 정보에서 좀 더 디테일하게 나오면 수집
dataset=[]

# 유튜브 정보를 담는 클래스(그릇역할)->향후 orm을 으로 연결되어 디비에 자동 삽입되는 형태로 변환
#BTS perform Idol!!! - BBC 게시자: BBC 2일 전 4분, 1초 조회수 1,965,185회

             
for video in getVideoElement():
    obj = YouTubeVideoModel()
    # 링크
    obj.link = video.find_element_by_id('thumbnail').get_attribute('href').split('=')[1]
    # 썸네일
    obj.thum = video.find_element_by_id('img').get_attribute('src')
    #나머지 정보
    other = [(a.text.strip(),a.get_attribute('href')) for a in video.find_elements_by_tag_name('a')]
    
    # 플레이시간
    obj.playtime = other[0][0]
    # 제목
    obj.title = other[1][0]
    # 작성자
    obj.author = other[2][0]
    #작성자 메인 페이지 링크
    
    obj.author_addr = other[2][1].split('/')[-1]
    print(obj)
    dataset.append(obj)
    
   
#    링크
#     print(video.find_element_by_id('thumbnail').get_attribute('href'))
    
#     # 썸네일
#     print('썸네일:',video.find_element_by_id('img').get_attribute('src'))    
    
        
#     # 플레이 시간, 제목, 작성자, 링크, 작성자 메인페이지 링크
    a_eles = video.find_elements_by_tag_name('a')
    print('재생시간',[(a.text.strip(),a.get_attribute('href')) for a in a_eles])
    
  


    
    # 조회수(억회,만회,천회,회)=> 숫자로 표현 
    # print( [span.text for span in video.find_elements_by_tag_name('span')][1]) # span 검색 한것은 이거 뿐인듯
    # 그러나 이건 무의미(버림 한결과 이므로)
    
    #print('조회수': video.find_element_by_css_selector('.style-scope ytd-video-meta-block').text.split('\n')[2])


        link =l89hLhtPOQY, 
        title=문 대통령, 한불 우정콘서트 관람…BTS 단 두 곡으로 무대 '완전장악' / 연합뉴스 (Yonhapnews), 
        author=연합뉴스 Yonhapnews, 
        author_addr=UCj3_t5p4L4aFsvdW3uHjnnw, 
        reviewCnt=0, 
        playtime=18:30, thum=https://i.ytimg.com/vi/l89hLhtPOQY/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDQPqaoXTfjmi8B1cqkTFbakjfCqA
        
재생시간 [('18:30', 'https://www.youtube.com/watch?v=l89hLhtPOQY'), ("문 대통령, 한불 우정콘서트 관람…BTS 단 두 곡으로 무대 '완전장악' / 연합뉴스 (Yonhapnews)", 'https://www.youtube.com/watch?v=l89hLhtPOQY'), ('연합뉴스 Yonhapnews', 'https://www.youtube.com/channel/UCj3_t5p4L4aFsvdW3uHjnnw')]

        link =ubKOuGRp3Z8, 
        title=DANCER REACTS TO BTS Bangtan Boys (방탄소년단) Random Play Dance, 
        author=Mr Okay, 
        author_addr=UCLVSSIjiAkpuoZXZGOeIgWw, 
        reviewCnt=0, 
        playtime=8:31, thum=https://i.ytimg.com/vi/ubKOuGRp3Z8/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCPWNbl9dBncPPHDyiwilsu7YKC

        
재생시간 [('', 'https://www.youtube.com/watch?v=M-z-mRDE7r0'), ('[BTS] 방탄소년단 & 대통령님과 찍은 사진의 개꿀잼 비하인드ㅋㅋㅋ (핵인싸 김석진', 'https://www.youtube.com/watch?v=M-z-mRDE7r0'), ('모니 모니', 'https://www.youtube.com/channel/UCyzhIbVGjin9o7RpmPGrtfQ')]

        link =9_MSAImEbe0, 
        title=[영상] BTS X 문재인 “이니 시계 싸인해주세요” 진 팬심 뿜뿜, 
        author=KBS News, 
        author_addr=NewsKBS, 
        reviewCnt=0, 
        playtime=, thum=None
        
재생시간 [('', 'https://www.youtube.com/watch?v=9_MSAImEbe0'), ('[영상] BTS X 문재인 “이니 시계 싸인해주세요” 진 팬심 뿜뿜', 'https://www.youtube.com/watch?v=9_MSAImEbe0'), ('KBS News', 'https://www.youtube.com/user/NewsKBS')]

        link =bZjSdBP1sZE, 
        title=BTS (日本語字幕) – ''FAKE LOVE' (Japanese Ver.) Lyrics [Color Coded_Kan_Rom_Eng], 
        author=Hamtaro Gasa, 
        author_addr=UC2EnwLe6j4GovEb8N3WZXBA, 
        reviewCnt=0, 
        playtime=, thum=None
        
재생시간 [('', 'https://www.youtube.com/watch?v=bZjSdBP1sZE'), ("BTS (日本語字幕) – ''FAKE LOVE' (Japanese Ver

        
재생시간 [('15:31', 'https://www.youtube.com/watch?v=VqAGP00WjG8'), ('문 대통령, 프랑스 국빈 방문 중 BTS와 특별한 만남 / SBS / 주영진의 뉴스브리핑', 'https://www.youtube.com/watch?v=VqAGP00WjG8'), ('SBS 뉴스', 'https://www.youtube.com/user/sbsnews8')]

        link =CSJwEM6vvk4, 
        title=VKOOK/TAEKOOK BTS OCTOBER 2018 100%REAL MOMENTS, 
        author=planetshipper, 
        author_addr=UCnckzBauxc1mOvajjrpII-w, 
        reviewCnt=0, 
        playtime=4:48, thum=https://i.ytimg.com/vi/CSJwEM6vvk4/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDzAL3AsNoxsqm3_ZMNwBanf2I4qw
        
재생시간 [('4:48', 'https://www.youtube.com/watch?v=CSJwEM6vvk4'), ('VKOOK/TAEKOOK BTS OCTOBER 2018 100%REAL MOMENTS', 'https://www.youtube.com/watch?v=CSJwEM6vvk4'), ('planetshipper', 'https://www.youtube.com/channel/UCnckzBauxc1mOvajjrpII-w')]

        link =ruDpXuxQ2q4, 
        title=181013 Love yourself tour in Amsterdam - Singularity / BTS V focus 4K fancam / 방탄소년단 뷔 직캠, 
        author=NUNA V, 
     

재생시간 [('10:09', 'https://www.youtube.com/watch?v=Y3G_sw3oSKM'), ('방탄소년단(BTS), 파리 깜짝 방문, 우정의 콘서트 @Paris (Korea-France Friendship Concert)', 'https://www.youtube.com/watch?v=Y3G_sw3oSKM'), ('Francezone Hanweekly', 'https://www.youtube.com/user/Hanweekly')]

        link =scFLjh16EFY, 
        title='준비된 자가 기회를 얻는다'… 문 대통령-BTS, 프랑스에서 만나 벌어진 일 / SBS, 
        author=SBS 뉴스, 
        author_addr=sbsnews8, 
        reviewCnt=0, 
        playtime=1:59, thum=https://i.ytimg.com/vi/scFLjh16EFY/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCx3TxOLHAU04WEnKELuzyrphzDTQ
        
재생시간 [('1:59', 'https://www.youtube.com/watch?v=scFLjh16EFY'), ("'준비된 자가 기회를 얻는다'… 문 대통령-BTS, 프랑스에서 만나 벌어진 일 / SBS", 'https://www.youtube.com/watch?v=scFLjh16EFY'), ('SBS 뉴스', 'https://www.youtube.com/user/sbsnews8')]

        link =3bLpxpuSQ4g, 
        title=【Nightcore】FAKE LOVE (Japanese Ver.) //BTS//, 
        author=TakotoNC - Nightcore Universe, 
        author_addr=UCTknq4m8fmiIUzvMVs0azSA,

재생시간 [('15:03', 'https://www.youtube.com/watch?v=92RQAM5peVg'), ('One Sweet Day | Cover | By Darly Ong , Khel, Bugoy, Katrina V + (BTS Footage) |MY REACTION', 'https://www.youtube.com/watch?v=92RQAM5peVg'), ('Jordan Smithy', 'https://www.youtube.com/user/olujimijordan12')]

        link =9beYpOniLBI, 
        title=BTS (방탄소년단) - 'FAKE LOVE Japanese ver ' (防弾少年団) FULL AUDIO, 
        author=BTS LOVE ARMY, 
        author_addr=UCO_FqLlVP9-QmkFD57eJpMw, 
        reviewCnt=0, 
        playtime=4:02, thum=https://i.ytimg.com/vi/9beYpOniLBI/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAvhRj8EHbXo0uugCxbRNb_R1au3g
        
재생시간 [('4:02', 'https://www.youtube.com/watch?v=9beYpOniLBI'), ("BTS (방탄소년단) - 'FAKE LOVE Japanese ver ' (防弾少年団) FULL AUDIO", 'https://www.youtube.com/watch?v=9beYpOniLBI'), ('BTS LOVE ARMY', 'https://www.youtube.com/channel/UCO_FqLlVP9-QmkFD57eJpMw')]

        link =VhzTbFcClQI, 
        title=BTS LOVE YOURSELF WORLD TOUR LONDON - MERCH HAUL!, 
 

...

        
재생시간 [('3:47', 'https://www.youtube.com/watch?v=HAYOR49aXzY'), ('Bets of me bts', 'https://www.youtube.com/watch?v=HAYOR49aXzY'), ('Dayana Diaz Vargas', 'https://www.youtube.com/channel/UCHRNutsPIjhV5AdIL9bg5Lg')]

        link =cJ3RVvWx_u0, 
        title=Bts - Magic piano tiles, 
        author=ITALO PRODUCCIONES, 
        author_addr=UCiSA-en6cmDKynEf52T6Z5Q, 
        reviewCnt=0, 
        playtime=3:13, thum=https://i.ytimg.com/vi/cJ3RVvWx_u0/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDbB0c_hP5U2hohFqswB6INrUbtZg
        
재생시간 [('3:13', 'https://www.youtube.com/watch?v=cJ3RVvWx_u0'), ('Bts - Magic piano tiles', 'https://www.youtube.com/watch?v=cJ3RVvWx_u0'), ('ITALO PRODUCCIONES', 'https://www.youtube.com/channel/UCiSA-en6cmDKynEf52T6Z5Q')]

        link =CWVb-A9FvEk, 
        title=Intenta no llorar #Army #BTS, 
        author=Imagina BTS & Mas, 
        author_addr=UC3Avqp98N3HBWTCeSgN4erA, 
        reviewCnt=0, 
        playtime=4:05, thum=h

재생시간 [('0:13', 'https://www.youtube.com/watch?v=H6hfCrYlO2M'), ('BTS Jungkook Moments with President', 'https://www.youtube.com/watch?v=H6hfCrYlO2M'), ('Red Suga', 'https://www.youtube.com/channel/UC02ezxDHUUj6AJqOl3N-AhQ')]

        link =JkWoT-cBQoA, 
        title=BTS (방탄소년단) - I Like It (좋아요) [Color Coded Han | Ita] [SUB ITA], 
        author=KookTae LoveYourself, 
        author_addr=UCPMfqBLxO0ZCTLH_VB8-tFA, 
        reviewCnt=0, 
        playtime=3:52, thum=https://i.ytimg.com/vi/JkWoT-cBQoA/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLA7Q9SJ-lwbrgoSp_wlUMq-Xm21PA
        
재생시간 [('3:52', 'https://www.youtube.com/watch?v=JkWoT-cBQoA'), ('BTS (방탄소년단) - I Like It (좋아요) [Color Coded Han | Ita] [SUB ITA]', 'https://www.youtube.com/watch?v=JkWoT-cBQoA'), ('KookTae LoveYourself', 'https://www.youtube.com/channel/UCPMfqBLxO0ZCTLH_VB8-tFA')]

        link =YqHxFajbcgM, 
        title=Making bts logo part 1, 
        author=J-money’s House of diys, 
        auth

        
재생시간 [('4:55', 'https://www.youtube.com/watch?v=2zKQjPy6uUA'), ('180915 Epiphany GA fancam @ BTS 방탄소년단 Love Yourself Tour in Fort Worth', 'https://www.youtube.com/watch?v=2zKQjPy6uUA'), ('bts_are_legendz', 'https://www.youtube.com/channel/UCQ55cryU9R4pUMMUyC8eKDQ')]

        link =76tcVZaMZn4, 
        title=BTS-DNA ORIGINAL VS BEST COVER, 
        author=Jaqueline Rueda, 
        author_addr=UCHBPIaBOksi2JOGFUJlKGVQ, 
        reviewCnt=0, 
        playtime=3:42, thum=https://i.ytimg.com/vi/76tcVZaMZn4/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCB8MR4KcroDEVRmpJSME6lWE9seQ
        
재생시간 [('3:42', 'https://www.youtube.com/watch?v=76tcVZaMZn4'), ('BTS-DNA ORIGINAL VS BEST COVER', 'https://www.youtube.com/watch?v=76tcVZaMZn4'), ('Jaqueline Rueda', 'https://www.youtube.com/channel/UCHBPIaBOksi2JOGFUJlKGVQ')]

        link =Bwy7Cn77icM, 
        title=Jin de BTS pide un autógrafo al presidente Moon Jae-in, 
        author=Kpop News, 
        author_addr

        
재생시간 [('9:28', 'https://www.youtube.com/watch?v=cX4g2U1N2ic'), ('BTS| 방탄소년단| (vlive) ENGLISH SUBS P3 Happy Birthday JIMIN’S!!!', 'https://www.youtube.com/watch?v=cX4g2U1N2ic'), ('ONLY K-POP JUST FOR YOU', 'https://www.youtube.com/channel/UCTV6Ir_i1ZMRAPfYHK5Zr7w')]

        link =4NA2inJWu1Y, 
        title="Enamorada de mi Hermanastro"~Taehyung y tú~/BTS HISTORIAS, 
        author=Alelovers, 
        author_addr=UCNODRZmUc61McE82QdD4BjA, 
        reviewCnt=0, 
        playtime=2:43, thum=https://i.ytimg.com/vi/4NA2inJWu1Y/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLBnn_4KnmTnaPVRs9EXLg5CewLl2Q
        
재생시간 [('2:43', 'https://www.youtube.com/watch?v=4NA2inJWu1Y'), ('"Enamorada de mi Hermanastro"~Taehyung y tú~/BTS HISTORIAS', 'https://www.youtube.com/watch?v=4NA2inJWu1Y'), ('Alelovers', 'https://www.youtube.com/channel/UCNODRZmUc61McE82QdD4BjA')]

        link =xWe0htyfvRU, 
        title=Guess the BTS song with emoji collab with Bella the BTS army

        
재생시간 [('3:55', 'https://www.youtube.com/watch?v=J9gT-2FfEPY'), ('اغنية Bts روووووووعة حماسية 💖❤️', 'https://www.youtube.com/watch?v=J9gT-2FfEPY'), ('anime anime', 'https://www.youtube.com/channel/UCD0ELwDoig0lfZME0QAF5aA')]

        link =jgg4V_lj6gE, 
        title=BTS: Bon Voyage; Season 3 Episode 2, 
        author=Nana Sagih 1, 
        author_addr=UCZkfYY64BWc4Av78RfTDVnQ, 
        reviewCnt=0, 
        playtime=59:32, thum=https://i.ytimg.com/vi/jgg4V_lj6gE/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCDWJ1_sx9iWGooFbD3IQuuf3py-A
        
재생시간 [('59:32', 'https://www.youtube.com/watch?v=jgg4V_lj6gE'), ('BTS: Bon Voyage; Season 3 Episode 2', 'https://www.youtube.com/watch?v=jgg4V_lj6gE'), ('Nana Sagih 1', 'https://www.youtube.com/channel/UCZkfYY64BWc4Av78RfTDVnQ')]

        link =ZUbwD2JtIJs, 
        title=BTS - BOY IN LUV 8D AUDIO (USE HEADPHONES), 
        author=Kim Yeontan, 
        author_addr=UCwJA4_rlm-4MCdq7_1vBXJA, 
        reviewCnt=0

In [28]:
# 숫자만 두고 나머지 문자는 제거 

def regNumber(src):
    filtering = filter(str.isdigit, src)
    if filtering:
        return ''.join(list(filtering)) # src의 문자하나하나 숫자냐 숫자가 아니냐
    else:
        return None
    

regNumber('조히수 129,303회')

'129303'

In [29]:
from tqdm import tqdm_notebook

In [31]:
# 세부 정도 획득
# 각 영상 페이지 방문 -> 정보 획득 -> 
# 댓글 데이터 획득(주기적으로 계속 획득해야함.) : 이미 수집한 사항에 대해서는 수집 x -> 중단 
# 댓글=> 시간 정렬 =>최신순으로 수집하면서 마지막 글 번호와 비교하여 그만 수집 

# tag를 만들어 쓰는게 angular(구글님들이 만드심) react임



for model in tqdm_notebook(dataset):
    url = 'https://www.youtube.com/watch?v=%s'% model.link
    print(url)
    # 세페이지 접속
    driver.get(url)
    time.sleep(5)  
    
# 게시일, 조회수, 좋아요, 싫어요, 영상 설명
    # 조회수
    countEle=driver.find_element_by_tag_name('yt-view-count-renderer')
    #print(countEle.find_element_by_tag_name('span').text)
    reg_count = regNumber(countEle.find_element_by_tag_name('span').text)
    print(reg_count)
#     for span in countEle.find_elements_by_tag_name('span'):
#         print(span.text)
    #print(driver.find_element_by_css_selector('.view-count style-scope yt-view-count-renderer'))
    
    # 좋아요, 싫어요
    # [None,좋아요 없음, 싫어요 없음,None,None]
    like_unlike = [ele.get_attribute('aria-label') 
           # 아이디가 text인 모든 요소를 찾아라.
           for ele in driver.find_elements_by_id('text') ]
           # aria-label 속성이 존재하는 요소만 모은다.
           #if ele.get_attribute('aria-label')] 
    print(type(like_unlike[0]))
    likes_cnt = regNumber(str(like_unlike[0]))
    if not likes_cnt:
        likes_cnt = 0
    hates_cnt = regNumber(str(like_unlike[1]))
    if not hates_cnt:
        hates_cnt = 0
    print(like_unlike)
    # 게시일
    regdate = driver.find_element_by_css_selector('#upload-info > span').text
    print(regdate )
    
    ## 영상설명    
    # 더보기 클릭
    try:
        driver.find_element_by_css_selector('#more yt-formatted-string').click()
    except Exception as e:pass
    # 영상설명    
    comment = driver.find_element_by_css_selector('#description > yt-formatted-string').text
    print('내용:',comment)
    # 카테고리
    category = driver.find_element_by_css_selector('#content > yt-formatted-string > a').text
    print('카테고리:',category)
    
    
    #조회수
    model.reviewCnt = reg_count
     # 게시일
    model.startDay = regdate
    # 좋아요 수
    model.likes = likes_cnt
    # 싫어요 수
    model.hates = hates_cnt
    # 영상 설명
    model.comment = comment
    # 카테고리
    model.cate = category
    
    
#댓글은 별도 
    # 작성자
    # 내용
    # 좋아요?
    #break



https://www.youtube.com/watch?v=l89hLhtPOQY
336861
<class 'str'>
['좋아요 7,590개', '싫어요 304개', None, None, None, None]
게시일: 2018. 10. 14.
내용: 프랑스를 국빈방문 중인 문재인 대통령은 14일(현지시각) 부인 김정숙 여사와 파리 트레지엄 아트 극장에서 열린 한·프랑스 문화교류 행사 '한국 음악의 울림-한불 우정의 콘서트'에 참석했다.
    
이날 콘서트는 문 대통령의 프랑스 방문을 계기로 양국 문화교류를 활성화하고 한국과 프랑스 간 우호적 분위기 조성을 위해 마련된 행사로, 특히 방탄소년단(BTS)의 출연이 결정되면서 문 대통령의 관람도 성사된 것으로 알려졌다.

영상취재 - [연합뉴스TV]


◆ 연합뉴스 홈페이지→ http://www.yna.co.kr/
◆ 이 시각 많이 본 기사 → https://goo.gl/VQTsSZ
◆ 오늘의 핫뉴스 → https://goo.gl/WyGXpG

◆ 현장영상 → https://goo.gl/5aZcx8
◆ 카드뉴스 →https://goo.gl/QKfDTH



◆연합뉴스 공식 SNS◆
◇페이스북→ https://www.facebook.com/yonhap/


▣ 연합뉴스 유튜브 채널 구독 : https://goo.gl/pL7TmT
▣ 연합뉴스 인스타 : https://goo.gl/UbqiQb
카테고리: 뉴스/정치
https://www.youtube.com/watch?v=ubKOuGRp3Z8
2279
<class 'str'>
['좋아요 230개', '싫어요 1개', None, None, None, None]
게시일: 2018. 10. 15.
내용: Hi peoples! Something fun to watch on a tired day :)

Follow me!
Twitter: https://twitter.com/MrOkayDance

vid i'm reacting to: https://www.youtube.com/wat

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"tag name","selector":"yt-view-count-renderer"}
  (Session info: chrome=69.0.3497.100)
  (Driver info: chromedriver=2.42.591088 (7b2b2dca23cca0862f674758c9a3933e685c27d5),platform=Windows NT 6.3.9600 x86_64)
